In [1]:
import pandas as pd
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('Twitter_Data.csv') 
df["clean_text"] = df["clean_text"].astype(str)

In [3]:
def sentiment_analysis_transformers(df):
    
    model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual"
    analyser = pipeline("sentiment-analysis", model=model_path)
    df['scores'] = df['clean_text'].apply(lambda text: analyser (text.lower()))
    df['Sentiment'] = df['scores'].apply(lambda output: output[0]['label']) 
    df['Sentiment'] = df['Sentiment'].apply(lambda x: 'Positive' if x == 'positive' else ('Negative' if x == 'negative' else 'Neutral')) 
    
    df =  df.drop(columns = ["scores"])
    return df

In [4]:
transformers_result = sentiment_analysis_transformers(df)

d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Load your CSV file
data = df

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to analyze sentiment and classify based on compound score
def classify_sentiment(text):
    # Get sentiment scores
    scores = analyzer.polarity_scores(text)
    
    # Get individual scores
    neg_score = scores['neg']
    neu_score = scores['neu']
    pos_score = scores['pos']
    compound_score = scores['compound']
    
    # Check if compound score is the highest
    if compound_score >= max(neg_score, neu_score, pos_score):
        # Classify based on compound score
        if compound_score >= 0.05:
            return "positive"
        elif compound_score > -0.05:
            return "neutral"
        else:
            return "negative"
    else:
        # Classify based on the highest individual score
        highest_score = max(neg_score, neu_score, pos_score)
        if highest_score == neg_score:
            return "negative"
        elif highest_score == neu_score:
            return "neutral"
        else:
            return "positive"
text = "This movie was okay, not great but not terrible"
print(classify_sentiment(text))

# # Apply the sentiment analysis function to the 'text' column
data['Sentiment'] = data['clean_text'].apply(classify_sentiment)
data
# Save the results to a new CSV file
# data.to_csv('data_with_sentiment.csv', index=False)




neutral


In [28]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load CSV file
# data = pd.read_csv('data.csv')

# Function to analyze sentiment
def analyze_sentiment(text):
    prompt = f"What is the sentiment of the following text? '{text}'"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=100, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Apply sentiment analysis to each row in the 'text' column
# data['sentiment'] = data['clean_text'].apply(analyze_sentiment)

# # Save the results to a new CSV file
# data.to_csv('data_with_sentiment.csv', index=False)

# print("Sentiment analysis complete! Results saved to 'data_with_sentiment.csv'.")
print(analyze_sentiment(text))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is the sentiment of the following text? 'This movie was okay, not great but not terrible'

I have been watching and looking at the movie because it's the same kind of movie that people should like. It's a movie with lots of character, lots of sex, big sex scenes, lots of nudity. They all have an emotional intensity to them, it's like a musical. When you look up to a big movie like this, it's the most emotional movie ever made
